In [1]:
import numpy as np
import pandas as pd
from pathlib import Path

pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 160)

TRADING_DAYS = 365

def _clean_numeric_series(s: pd.Series) -> pd.Series:
    if s.dtype == "O":
        s2 = (
            s.astype(str)
             .str.replace(",", "", regex=False)
             .str.replace("$", "", regex=False)
             .str.replace("%", "", regex=False)
             .str.strip()
        )
        return pd.to_numeric(s2, errors="coerce")
    return pd.to_numeric(s, errors="coerce")

def _find_first_col(df: pd.DataFrame, candidates):
    cols = {c.lower(): c for c in df.columns}
    for cand in candidates:
        if cand.lower() in cols:
            return cols[cand.lower()]
    return None

def read_price_csv(path: Path) -> pd.DataFrame:
    df = pd.read_csv(path)
    date_col = _find_first_col(df, ["End", "Start", "Date", "timestamp", "Datetime", "Time"])
    if date_col is None:
        raise ValueError(f"Cannot find a date column in {path.name}. Columns: {list(df.columns)}")
    df[date_col] = pd.to_datetime(df[date_col], errors="coerce", infer_datetime_format=True)
    df = df.dropna(subset=[date_col]).sort_values(date_col).set_index(date_col)

    for c in df.columns:
        df[c] = _clean_numeric_series(df[c]) if df[c].dtype == "O" else df[c]

    close_col = _find_first_col(df, ["Close", "Price", "Adj Close", "AdjClose"])
    if close_col is None:
        raise ValueError(f"Cannot find a close/price column in {path.name}. Columns: {list(df.columns)}")

    if "Close" not in df.columns:
        df["Close"] = df[close_col].astype(float)

    return df

def to_daily_close(df: pd.DataFrame) -> pd.Series:
    s = df["Close"].copy()
    s = s[~s.index.duplicated(keep="last")].sort_index()
    s = s.asfreq("D").ffill()
    return s

def sharpe_ratio(daily_returns: pd.Series) -> float:
    r = daily_returns.dropna()
    if len(r) < 2:
        return np.nan
    vol = r.std(ddof=1)
    if vol == 0 or np.isnan(vol):
        return np.nan
    return (r.mean() / vol) * np.sqrt(TRADING_DAYS)


In [2]:
def read_price_csv(path: Path):
    df = pd.read_csv(path)

    # find date column
    for col in ["Date", "End", "Start", "timestamp", "Timestamp"]:
        if col in df.columns:
            date_col = col
            break
    else:
        raise ValueError(f"No date column found in {path.name}. Columns: {list(df.columns)}")

    # FIX: removed infer_datetime_format
    df[date_col] = pd.to_datetime(df[date_col], errors="coerce")

    df = df.dropna(subset=[date_col]).sort_values(date_col).set_index(date_col)

    # IBIT uses Price instead of Close
    if "Close" not in df.columns:
        if "Price" in df.columns:
            df["Close"] = df["Price"]
        else:
            raise ValueError(f"No Close/Price column found in {path.name}")

    # clean numeric columns
    for c in df.columns:
        if df[c].dtype == "O":
            df[c] = (
                df[c]
                .astype(str)
                .str.replace(",", "", regex=False)
                .str.replace("$", "", regex=False)
                .str.replace("%", "", regex=False)
                .str.strip()
            )
            df[c] = pd.to_numeric(df[c], errors="ignore")

    return df


In [6]:
import numpy as np
import pandas as pd
from pathlib import Path

# --- point to your folder (same one that worked before) ---
DATA_DIR = Path.home() / "Desktop" / "Quant Data" / "Cryptocurrency"
# If needed, use iCloud Desktop instead:
# DATA_DIR = Path.home() / "Library" / "Mobile Documents" / "com~apple~CloudDocs" / "Desktop" / "Quant Data" / "Cryptocurrency"

BTC_PATH  = DATA_DIR / "BTC.csv"
ETHA_PATH = DATA_DIR / "ETHA.csv"

def read_price_csv_simple(path: Path) -> pd.DataFrame:
    if not path.exists():
        raise FileNotFoundError(f"File not found: {path}")

    df = pd.read_csv(path)

    # date col
    for col in ["End", "Start", "Date", "timestamp", "Timestamp"]:
        if col in df.columns:
            date_col = col
            break
    else:
        raise ValueError(f"No date column in {path.name}. Columns: {list(df.columns)}")

    df[date_col] = pd.to_datetime(df[date_col], errors="coerce")
    df = df.dropna(subset=[date_col]).sort_values(date_col).set_index(date_col)

    # close col
    if "Close" not in df.columns:
        if "Price" in df.columns:
            df["Close"] = df["Price"]
        else:
            raise ValueError(f"No Close/Price in {path.name}. Columns: {list(df.columns)}")

    # numeric cleanup
    for c in df.columns:
        if df[c].dtype == "O":
            df[c] = (
                df[c].astype(str)
                    .str.replace(",", "", regex=False)
                    .str.replace("$", "", regex=False)
                    .str.replace("%", "", regex=False)
                    .str.strip()
            )
            df[c] = pd.to_numeric(df[c], errors="coerce")

    return df

def to_daily_close(df: pd.DataFrame) -> pd.Series:
    s = df["Close"].astype(float).copy()
    s = s[~s.index.duplicated(keep="last")].sort_index()
    return s.asfreq("D").ffill()

def sharpe_ratio(daily_returns: pd.Series, trading_days: int = 365) -> float:
    r = daily_returns.dropna()
    if len(r) < 2:
        return np.nan
    vol = r.std(ddof=1)
    if vol == 0 or np.isnan(vol):
        return np.nan
    return (r.mean() / vol) * np.sqrt(trading_days)

# --- load + build ratio df ---
btc_df = read_price_csv_simple(BTC_PATH)
etha_df = read_price_csv_simple(ETHA_PATH)

btc_close = to_daily_close(btc_df)
etha_close = to_daily_close(etha_df)

ratio = (btc_close / etha_close).rename("btc_over_etha")
df = pd.DataFrame({"btc": btc_close, "etha": etha_close, "ratio": ratio}).dropna()

# --- strategy: long BTC when ratio is "low" vs its rolling mean ---
def band_signal_ratio_long_btc(ratio: pd.Series, window: int, z_entry: float = 1.0, z_exit: float = 0.25) -> pd.Series:
    log_r = np.log(ratio)
    m = log_r.rolling(window, min_periods=window).mean()
    s = log_r.rolling(window, min_periods=window).std(ddof=1)
    z = (log_r - m) / s

    pos = pd.Series(0.0, index=ratio.index)
    in_pos = False

    for i in range(len(pos)):
        zi = z.iat[i]
        if np.isnan(zi):
            pos.iat[i] = pos.iat[i-1] if i > 0 else 0.0
            continue

        if (not in_pos) and (zi < -z_entry):
            in_pos = True
        elif in_pos and (zi > -z_exit):
            in_pos = False

        pos.iat[i] = 1.0 if in_pos else 0.0

    return pos

pos_ratio = band_signal_ratio_long_btc(df["ratio"], window=90, z_entry=1.0, z_exit=0.25)

btc_ret = df["btc"].pct_change()

strat_ret = pos_ratio.shift(1) * btc_ret
bh_ret = btc_ret

print("Ratio model Sharpe (long BTC only):", sharpe_ratio(strat_ret))
print("BTC buy&hold Sharpe:", sharpe_ratio(bh_ret))

df.tail()


Ratio model Sharpe (long BTC only): 0.03470975152245687
BTC buy&hold Sharpe: 1.0826030758766478


,btc,etha,ratio
End,,,
2026-02-06,62951.51,1824.6902,34.499835
2026-02-07,70556.88,2061.3085,34.229170
2026-02-08,69341.96,2090.9076,33.163570
2026-02-09,70580.10,2096.1317,33.671596
2026-02-10,70162.10,2105.0785,33.329921
